In [15]:
import pandas as pd
import numpy as np

file_path = "cs_bisnode_panel.csv"
df = pd.read_csv(file_path)

info = df.info()
head = df.head()
columns = df.columns.tolist()

# Describe relevant numeric columns only
desc = df.describe(include='all')

df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287829 entries, 0 to 287828
Data columns (total 48 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   comp_id               287829 non-null  float64
 1   begin                 287829 non-null  object 
 2   end                   287829 non-null  object 
 3   COGS                  18257 non-null   float64
 4   amort                 279789 non-null  float64
 5   curr_assets           287698 non-null  float64
 6   curr_liab             287698 non-null  float64
 7   extra_exp             269300 non-null  float64
 8   extra_inc             269300 non-null  float64
 9   extra_profit_loss     270626 non-null  float64
 10  finished_prod         17485 non-null   float64
 11  fixed_assets          287698 non-null  float64
 12  inc_bef_tax           280392 non-null  float64
 13  intang_assets         287689 non-null  float64
 14  inventories           287698 non-null  float64
 15  

,comp_id,begin,end,COGS,amort,curr_assets,curr_liab,extra_exp,extra_inc,extra_profit_loss,...,gender,origin,nace_main,ind2,ind,urban_m,region_m,founded_date,exit_date,labor_avg
0,1.001034e+06,2005-01-01,2005-12-31,NaN,692.592590,7266.666504,7574.074219,0.0,0.0,0.0,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,NaN
1,1.001034e+06,2006-01-01,2006-12-31,NaN,603.703674,13122.222656,12211.111328,0.0,0.0,0.0,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,NaN
2,1.001034e+06,2007-01-01,2007-12-31,NaN,425.925934,8196.295898,7800.000000,0.0,0.0,0.0,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,NaN
3,1.001034e+06,2008-01-01,2008-12-31,NaN,300.000000,8485.185547,7781.481445,0.0,0.0,0.0,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,NaN
4,1.001034e+06,2009-01-01,2009-12-31,NaN,207.407410,5137.037109,15300.000000,0.0,0.0,0.0,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,0.083333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287824,4.641209e+11,2011-01-01,2011-12-31,NaN,0.000000,1807.407349,3.703704,0.0,0.0,0.0,...,female,Domestic,5510.0,55.0,3.0,2,East,1990-08-22,NaN,NaN
287825,4.641209e+11,2012-01-01,2012-12-31,NaN,0.000000,1518.518555,96.296295,0.0,0.0,0.0,...,female,Domestic,5510.0,55.0,3.0,2,East,1990-08-22,NaN,NaN
287826,4.641209e+11,2013-01-01,2013-12-31,NaN,0.000000,988.888916,77.777779,0.0,0.0,0.0,...,NaN,NaN,5510.0,55.0,3.0,2,East,1990-08-22,NaN,NaN
287827,4.641209e+11,2014-01-01,2014-12-31,NaN,0.000000,644.444458,233.333328,0.0,0.0,0.0,...,female,Domestic,5510.0,55.0,3.0,2,East,1990-08-22,NaN,NaN


In [16]:
df = df.drop(
    columns=["COGS", "finished_prod", "net_dom_sales", "net_exp_sales", "wages"]
)
df = df.query("year !=2016")



In [17]:
df = (
    df.set_index(["year", "comp_id"])
    .unstack(fill_value="toReplace")
    .stack()
    .reset_index()
)
df = df.replace("toReplace", np.nan)  

/var/folders/6n/p578p__s54j_ngsncpgm562m0000gn/T/ipykernel_13335/3602686008.py:4: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  .stack()
/var/folders/6n/p578p__s54j_ngsncpgm562m0000gn/T/ipykernel_13335/3602686008.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace("toReplace", np.nan)


In [8]:
df

,year,comp_id,begin,end,amort,curr_assets,curr_liab,extra_exp,extra_inc,extra_profit_loss,...,gender,origin,nace_main,ind2,ind,urban_m,region_m,founded_date,exit_date,labor_avg
0,2005,1.001034e+06,2005-01-01,2005-12-31,692.592590,7.266667e+03,7.574074e+03,0.000000,0.000000,0.000000,...,mix,Domestic,5630.0,56.0,3.0,1.0,Central,1990-11-19,NaN,NaN
1,2005,1.001541e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2005,1.002029e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2005,1.003200e+06,2005-01-01,2005-12-31,25.925926,7.296296e+02,3.803704e+03,0.000000,4814.814941,4814.814941,...,NaN,NaN,5630.0,56.0,3.0,1.0,Central,2003-10-21,2014-08-09,NaN
4,2005,1.004012e+06,2005-01-01,2005-12-31,114.814812,1.648148e+03,3.777778e+03,0.000000,0.000000,0.000000,...,male,Domestic,3314.0,33.0,2.0,3.0,West,1991-11-24,2013-06-19,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510527,2015,4.640446e+11,2015-01-01,2015-12-31,83425.929688,6.532030e+06,2.253478e+06,0.000000,0.000000,0.000000,...,male,Foreign,3312.0,33.0,2.0,3.0,Central,2008-10-09,NaN,3.138889
510528,2015,4.640568e+11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
510529,2015,4.640793e+11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
510530,2015,4.641050e+11,2015-01-01,2015-12-31,1288.888916,6.638519e+04,1.375185e+04,18.518518,0.000000,-18.518518,...,mix,Domestic,5610.0,56.0,3.0,3.0,West,1989-04-28,NaN,1.138889


In [ ]:
df["status_alive"] = (df["sales"] > 0 & (False == df["sales"].isna())).astype(int)


df["default"] = (
    (df["status_alive"] == 1)
    & (df.groupby("comp_id")["status_alive"].shift(-2) == 0)
).astype(int)


In [12]:
df = df[(df["year"] >= 2010) & (df["year"] <= 2015)]


In [13]:
df = df.drop(columns=["COGS", "finished_prod", "net_dom_sales", "net_exp_sales", "wages"], errors='ignore')


In [ ]:
df["sales"].describe()


In [ ]:
df["sales"] = np.where(
    df["sales"] < 0, 1, df["sales"]
)  # has to be outside of assin()
df = df.assign(
    ln_sales=np.where(
        df["sales"] > 0,
        np.log(df["sales"]),
        (np.where(df["sales"].isna(), np.nan, 0)),
    ),  # NaN remain NaN
    sales_mil=df["sales"] / 1000000,
    sales_mil_log=np.where(
        df["sales"] > 0,
        np.log(df["sales"] / 1000000),
        (np.where(df["sales"].isna(), np.nan, 0)),
    ),
)

In [ ]:
df["d1_sales_mil_log"] = df["sales_mil_log"] - df.groupby("comp_id")[
    "sales_mil_log"
].shift(1)

In [ ]:
# replace w 0 for new firms + add dummy to capture it
df["age"] = np.where(
    df["year"] - df["founded_year"] < 0, 0, df["year"] - df["founded_year"]
)
df["new"] = np.where(
    ((df["age"] <= 1) | (df["balsheet_notfullyear"] == 1)),
    1,
    (np.where(df["age"].isna(), np.nan, 0)),
)
df["d1_sales_mil_log"] = np.where(
    df["new"] == 1, 0, np.where(df["new"].isna(), np.nan, df["d1_sales_mil_log"])
)
df["new"] = np.where(df["d1_sales_mil_log"].isna(), 1, df["new"])
df["d1_sales_mil_log"] = np.where(
    df["d1_sales_mil_log"].isna(), 0, df["d1_sales_mil_log"]
)